In [0]:
!pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 33kB/s 


In [0]:
import tensorflow as tf 
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
tf.__version__

'2.2.0'

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6836596540844802070
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7096132556027109138
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7639457061571691445
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15788201792
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7436455614293531966
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [0]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
sess=tf.compat.v1.InteractiveSession()

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':'1vmDB7CBUMF2gGH2D_EuSDmSaCiJxDJv6'}) 
downloaded.GetContentFile('ratings.csv')  

In [0]:
directory_name = ''
filename = os.path.join(directory_name, 'ratings.csv')
dataset = pd.read_csv(filename, sep=',')

X_Data = np.asanyarray(dataset.sort_values(['userId', 'timestamp']).groupby('userId')['timestamp'].apply(list))
X_data = dataset.sort_values(['userId', 'timestamp']).groupby('userId')['movieId'].apply(list)
X_data = tf.keras.preprocessing.sequence.pad_sequences(X_data)

l = []
for i in range(1, len(X_data)):
    l1 = []
    for e in X_data[i]:
        l1.append(str(e))
    l.append(l1)

In [0]:
len(l)

7907

In [0]:
word_model = Word2Vec(l, size=100, min_count=1, window=5, iter=10)
word_model.save('word2vec.model')
# noinspection PyTypeChecker
max_sentence_len = len(max(l, key=len))


def word2idx(word_):
    return word_model.wv.vocab[word_].index


def idx2word(idx):
    return word_model.wv.index2word[idx]


train_x = np.zeros([len(l), max_sentence_len, 100], dtype=np.float32)
train_y = np.zeros([len(l), 100], dtype=np.float32)
for i, sentence in enumerate(l):
    for t, word in enumerate(sentence[:-1]):
        train_x[i, t] = word_model[word]
    train_y[i] = word_model[word]

In [0]:
type(word_model['1'][0])

numpy.float32

In [0]:
vocab_size = len(word_model.wv.vocab)
embedding_size = 200

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_size))
model.add(tf.keras.layers.LSTM(units=embedding_size))
model.add(tf.keras.layers.Dense(units=vocab_size))
model.add(tf.keras.layers.Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.summary()
history = model.fit(train_x, train_y, epochs=1)

# pd.Series(history.history['loss']).plot(logy=True)
# plt.xlabel("Epoch")
# plt.ylabel("Train Error")

model_save_path = os.path.join(directory_name, 'keras_model.h5')
model.save(model_save_path)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         2871400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense (Dense)                (None, 14357)             2885757   
_________________________________________________________________
activation (Activation)      (None, 14357)             0         
Total params: 6,077,957
Trainable params: 6,077,957
Non-trainable params: 0
_________________________________________________________________
248/248 [==============================] - 66s 266ms/step - loss: 8.7620


In [0]:
inputs = Input(batch_shape=(args.batch_size, 1, args.train_n_items))
    gru, gru_states = CuDNNGRU(hidden_units, stateful=True, return_state=True)(inputs)
    drop2 = Dropout(0.25)(gru)
    predictions = Dense(args.train_n_items, activation='softmax')(drop2)
    model = Model(input=inputs, output=[predictions])
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=categorical_crossentropy, optimizer=opt)
    model.summary()